In [ ]:
# default_exp diff_analysis_manager

In [ ]:
from ms_empire.background_distributions import *
from ms_empire.normalization import *
from ms_empire.diff_analysis import *

In [ ]:
#export
#read in peptide datafiles



In [ ]:
#export
import pandas as pd
from itertools import combinations
import numpy as np

def run_pipeline(unnormed_df, labelmap_df, minrep):
    conds = labelmap_df["condition"].unique()
    condpair2diffions = {}
    for condpair in combinations(conds, 2):
        
        c1_samples = labelmap_df[labelmap_df["condition"]== condpair[0]]
        c2_samples = labelmap_df[labelmap_df["condition"]== condpair[1]]
        df_c1 = unnormed_df.loc[:, c1_samples["sample"]].dropna(thresh=c1_samples.shape[0]-minrep, axis=0)
        df_c2 = unnormed_df.loc[:, c2_samples["sample"]].dropna(thresh=c2_samples.shape[0]-minrep, axis=0)
        display(df_c1)
        df_c1_normed = pd.DataFrame(normalize_withincond(df_c1.to_numpy().T).T, index = df_c1.index)
        print("end norm1")
        display(df_c1_normed)
        df_c2_normed = pd.DataFrame(normalize_withincond(df_c2.to_numpy().T).T, index = df_c2.index)
        
        normed_c1 = ConditionBackgrounds(df_c1_normed)
        normed_c2 = ConditionBackgrounds(df_c2_normed)

        ions_to_check = normed_c1.ion2nonNanvals.keys() & normed_c2.ion2nonNanvals.keys()

        ion2diffDist = {}
        diffIons = []
        for ion in ions_to_check:
            vals1 = normed_c1.ion2nonNanvals.get(ion)
            vals2 = normed_c2.ion2nonNanvals.get(ion)
            bgdist1 = normed_c1.ion2background.get(ion)
            bgdist2 = normed_c2.ion2background.get(ion)
            diffDist = get_subtracted_bg(ion2diffDist,bgdist1, bgdist2,ion )
            diffIon = DifferentialIon(vals1, vals2, diffDist)
            diffIons.append(diffIon)
        
        condpair2diffions.update({condpair :diffIon})
        


In [ ]:
#hide
import pandas as pd
import numpy as np
import uuid

def test_run_pipeline():

    sample2cond_df = pd.DataFrame({'sample' : ['A1', 'A2', 'A3', 'B1', 'B2', 'B3'], 
    'condition' : ['A', 'A', 'A', 'B', 'B', 'B']})
    unnormed_df = generate_random_input(1000, sample2cond_df)
    print(unnormed_df)
    run_pipeline(unnormed_df, sample2cond_df, 3)


def generate_random_input(num_pep,sample2cond_df ):
    pepnames = list(map(lambda _idx : str(uuid.uuid4()), range(num_pep))) #gives uuid strings for each peptide
    randarrays = 10+ 1.5*np.random.randn(num_pep,sample2cond_df.shape[0])
    df_intens = pd.DataFrame(randarrays, columns= sample2cond_df["sample"].tolist())
    df_intens.insert(0, "peptides", pepnames )
    df_intens = df_intens.set_index("peptides")
    return df_intens

test_run_pipeline()


                                             A1         A2         A3  \
peptides                                                                
240118f4-d894-4d5a-8df2-c545a698b58c  11.508772   9.931643   7.498094   
65a10810-2c7d-4dcb-a264-5d92968b1a1e   9.963546   9.052812   8.712299   
19f5fad0-d668-4d2f-9ecc-4efae2aac92f   8.795783   8.848097   7.862067   
867c80a3-c175-4f99-b82a-a417fc4ffecc  10.757026  10.036202  10.467125   
a3d88277-9944-4f03-b15e-aa4148bd2d5d  13.462107  13.183907  11.248265   
...                                         ...        ...        ...   
13df913a-143b-432e-a22a-d3829e8d0735   9.020774   9.940004  10.894677   
4e64433d-148f-415b-8104-35a647c414fd  10.987855  11.033567   8.715090   
0a999243-dbd1-42f5-846b-b1dc1301b364  10.724362  11.630650   9.870438   
d8ec2852-b189-4914-9ffc-e8bd355b3c78  12.428930   8.621795   9.129174   
8cf8abf2-dc7b-4848-abd7-fa0e97ee8d38  10.863791   7.399682   9.850692   

                                             B1   

,A1,A2,A3
peptides,,,
240118f4-d894-4d5a-8df2-c545a698b58c,11.508772,9.931643,7.498094
65a10810-2c7d-4dcb-a264-5d92968b1a1e,9.963546,9.052812,8.712299
19f5fad0-d668-4d2f-9ecc-4efae2aac92f,8.795783,8.848097,7.862067
867c80a3-c175-4f99-b82a-a417fc4ffecc,10.757026,10.036202,10.467125
a3d88277-9944-4f03-b15e-aa4148bd2d5d,13.462107,13.183907,11.248265
...,...,...,...
13df913a-143b-432e-a22a-d3829e8d0735,9.020774,9.940004,10.894677
4e64433d-148f-415b-8104-35a647c414fd,10.987855,11.033567,8.715090
0a999243-dbd1-42f5-846b-b1dc1301b364,10.724362,11.630650,9.870438


end norm1


,0,1,2
peptides,,,
240118f4-d894-4d5a-8df2-c545a698b58c,11.508772,9.846087,7.333303
65a10810-2c7d-4dcb-a264-5d92968b1a1e,9.963546,8.967255,8.547508
19f5fad0-d668-4d2f-9ecc-4efae2aac92f,8.795783,8.762540,7.697276
867c80a3-c175-4f99-b82a-a417fc4ffecc,10.757026,9.950645,10.302334
a3d88277-9944-4f03-b15e-aa4148bd2d5d,13.462107,13.098350,11.083475
...,...,...,...
13df913a-143b-432e-a22a-d3829e8d0735,9.020774,9.854448,10.729886
4e64433d-148f-415b-8104-35a647c414fd,10.987855,10.948011,8.550299
0a999243-dbd1-42f5-846b-b1dc1301b364,10.724362,11.545093,9.705648


7c': array([10.81245085, 10.21602817,  9.58417215]), '55d8bf2c-2c77-4a75-95ef-94b371168f24': array([10.54911389, 10.01012594,  9.56924366]), 'eda92f8a-1898-42c5-86e7-135a82d016b5': array([9.50465583, 8.4305094 , 9.10814507]), 'b1c22afd-93db-4d78-92e4-677e6dcab922': array([10.4808081 ,  9.31868845,  9.18681649]), 'c89402ad-b0bd-431d-a694-df093787e00e': array([10.69339211,  9.78023189, 10.7762606 ]), 'b9fce8f4-eca6-46db-8b11-afc9241ea0c7': array([11.05535223,  6.66328779, 11.06266548]), 'e28f845a-5117-4668-9a84-09ea06a9505e': array([10.06799815,  9.48625747, 10.22616431]), '1692f791-cf87-4c01-aa57-721b1b79ce02': array([ 9.94435676,  8.54344858, 11.99104718]), '539b2392-78c8-48fc-899d-99d2ce8249aa': array([10.53294671, 10.48102758,  7.72636054]), '75dfe0a5-cadb-4dad-b203-65dea49430d8': array([ 9.66943615,  9.37393185, 10.57062216]), '7a2d2852-523f-4b05-be7a-58d6899ff03c': array([ 9.89126931, 11.42186921, 10.71827513]), '428d6886-ee12-4745-b475-6e618f944626': array([10.65170818,  8.1958945

KeyError: 0